In [49]:
import pandas as pd
import numpy as np
from scipy.stats import gmean

In [50]:
grampa = pd.read_csv("origin_data/grampa_v2.csv").drop("Unnamed: 0", axis=1)
uniprot = pd.read_csv("origin_data/origin_negative.csv")

In [51]:
# grampa = grampa[grampa["bacterium"] == "E. coli"]
# grampa = grampa.assign(MIC = lambda x: 10**x['value'])
# grampa["sequence"] = grampa.apply(lambda row: row['sequence'] + "_amd" if row["has_cterminal_amidation"] else row["sequence"], axis=1)
# grampa = grampa[~grampa["has_unusual_modification"]]

# grampa = grampa.groupby("sequence").agg({"MIC": gmean}).reset_index()

# grampa = grampa.rename(columns={'MIC': 'MIC_ecoli'})
# grampa["MIC_ecoli"].describe()


### Uniprot:

In [52]:
uniprot = uniprot.rename(columns={"Sequence": "sequence"})
uniprot = uniprot[~uniprot["sequence"].str.contains("B|X|Z|O|U")].reset_index(drop=True)
uniprot["Keywords"] = uniprot["Keywords"].fillna("")

uniprot["sequence"] = uniprot.apply(lambda row: row['sequence'] + "_amd" if "amidation" in row["Keywords"].lower() else row["sequence"], axis=1)

# uniprot[uniprot["Sequence"].duplicated(keep=False)].groupby("Sequence").size().reset_index(name='counts')
uniprot = uniprot[~uniprot["sequence"].duplicated()].reset_index(drop=True)

uniprot["MIC_ecoli"] = [9999] * len(uniprot)

uniprot.to_csv("formatted_data/uniprot.csv", columns = ["sequence", "MIC_ecoli"], index=False)

### GRAMPA:

In [53]:
grampa = grampa[grampa["bacterium"] == "E. coli"]
grampa = grampa.assign(MIC = lambda x: 10**x['value'])
grampa["sequence"] = grampa.apply(lambda row: row['sequence'] + "_amd" if row["has_cterminal_amidation"] else row["sequence"], axis=1)
grampa = grampa[~grampa["has_unusual_modification"]]

grampa = grampa.groupby("sequence").agg({"MIC": gmean}).reset_index()

grampa = grampa.rename(columns={'MIC': 'MIC_ecoli'})

grampa.to_csv("formatted_data/grampa.csv", columns = ["sequence", "MIC_ecoli"], index=False)

### Concatenate:

In [70]:
df = pd.concat([grampa, uniprot], ignore_index=True).dropna(axis=1)
df = df[df["sequence"].duplicated(keep=False) == False]

df.insert(0, 'pep_ID', range(1, len(df) + 1))

df.to_csv('peptide_db.csv', index=False)
df

,pep_ID,sequence,MIC_ecoli
0,1,AAAAAAAAAAGIGKFLHSAKKFGKAFVGEIMNS,125.878150
1,2,AAAAAAAIKMLMDLVNERIMALNKKAKK_amd,10.000000
2,3,AAAAGSVWGAVNYTSDCNGECKRRGYKGGYCGSFANVNCWCET,100.000000
3,4,AAAKAALNAVLVGANA,80.000000
4,5,AACSDRAHGHICESFKSFCKDSGRNGVKLRANCKKTCGLC,1.780176
...,...,...,...
11066,10999,IFFEV,9999.000000
11067,11000,FAIGAIIKKFGKKLVEYAVKHKDLYAPYIKKHLCEKL,9999.000000
11068,11001,SDQGDVAEPKMHKTAPPFDFEAIPEEYLDDES,9999.000000
11069,11002,MMYLMPLLIVIVGCLALHCSDDDMPDGHA,9999.000000
